<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [128]:
!pip3 install beautifulsoup4
!pip3 install requests

Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [129]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [130]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [131]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [132]:
# use requests.get() method with the provided static_url
# assign the response to a object
response = requests.get(static_url)
response.status_code

200

Create a `BeautifulSoup` object from the HTML `response`


In [133]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
response_BS = BeautifulSoup(response.text,'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [134]:
# Use soup.title attribute
response_BS.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [135]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = response_BS.find_all('table')

Starting from the third table is our target table contains the actual launch records.


In [136]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [137]:
# Initialize an empty list to store column names
column_names = []

# Find all <th> elements in the first_launch_table
# `find_all()` returns a ResultSet, which is a list of all matching elements
th_ele = first_launch_table.find_all('th')

# Print the ResultSet to verify the extracted <th> elements
print(th_ele)

# Iterate through each <th> element and its index in the ResultSet
for i, th in enumerate(th_ele):
    # Print the index for debugging purposes
    print(i)

    # Call the extract_column_from_header function with the current <th> element
    # This function is designed to process a single HTML element to extract a column name
    column_nm = extract_column_from_header(th)

    # Print the extracted column name for debugging
    print(column_nm)

    # Check if the column name is not None and is not an empty string
    # `len(column_nm) > 0` ensures that the column name has content
    if column_nm is not None and len(column_nm) > 0:
        # Append the non-empty column name to the column_names list
        column_names.append(column_nm)

# Explanation of code:
# 1. `find_all('th')` retrieves all <th> elements from the table. This is a ResultSet, not a single element.
# 2. Iterate over each element in `th_ele` using a for loop. 
# 3. `extract_column_from_header(th)` processes each <th> element to extract and clean the column name.
# 4. Print the index and extracted column name to debug and verify the results.
# 5. Append only non-empty column names (those that are not None and have length > 0) to the `column_names` list.

[<th scope="col">Flight No.
</th>, <th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>, <th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>, <th scope="col">Launch site
</th>, <th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>, <th scope="col">Payload mass
</th>, <th scope="col">Orbit
</th>, <th scope="col">Customer
</th>, <th scope="col">Launch<br/>outcome
</th>, <th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th>, <th rowspan="2" s

Check the extracted column names


In [138]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [139]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [140]:
# Initialize a counter for the number of extracted rows
extracted_row = 0

# Iterate over each table found in the HTML content
for table_number, table in enumerate(html_tables):
    # Iterate over each row in the current table
    for rows in table.find_all("tr"):
        # Check if the row has a <th> element and if it contains a valid flight number
        if rows.th and rows.th.string:
            # Extract and clean the flight number
            flight_number = rows.th.string.strip()
            # Check if the flight number is numeric
            flag = flight_number.isdigit()
        else:
            flag = False

        # Get all <td> elements in the current row
        row = rows.find_all('td')

        # Process the row if it's determined to be valid (i.e., it contains a flight number)
        if flag:
            # Increment the count of extracted rows
            extracted_row += 1

            # Extract and append the Flight Number to the launch_dict
            flight_number = rows.th.string.strip() if rows.th else None
            if flight_number:
                launch_dict['Flight No.'].append(flight_number)

            # Extract and process Date and Time from the first cell
            # The function date_time() should return a list with [date, time]
            datatimelist = date_time(row[0])  # Process the first <td> element
            # Append Date and Time to the launch_dict
            launch_dict['Date'].append(datatimelist[0].strip(','))
            launch_dict['Time'].append(datatimelist[1])

            # Extract and process Booster Version from the second cell
            # The function booster_version() should handle this extraction
            bv = booster_version(row[1])  # Process the second <td> element
            # If booster_version() returns None, use the text in <a> tag if present
            if not bv and row[1].a:
                bv = row[1].a.string
            # Append Booster Version to the launch_dict
            launch_dict['Version Booster'].append(bv if bv else 'Unknown')

            # Extract and process Launch Site from the third cell
            # Use the text in <a> tag if present
            launch_site = row[2].a.string if row[2].a else 'Unknown'
            # Append Launch Site to the launch_dict
            launch_dict['Launch site'].append(launch_site)

            # Extract and process Payload from the fourth cell
            # Use the text in <a> tag if present
            payload = row[3].a.string if row[3].a else 'Unknown'
            # Append Payload to the launch_dict
            launch_dict['Payload'].append(payload)

            # Extract and process Payload Mass from the fifth cell
            # The function get_mass() should handle this extraction
            payload_mass = get_mass(row[4])  # Process the fifth <td> element
            # Append Payload Mass to the launch_dict
            launch_dict['Payload mass'].append(payload_mass if payload_mass else 'Unknown')

            # Extract and process Orbit from the sixth cell
            # Use the text in <a> tag if present
            orbit = row[5].a.string if row[5].a else 'Unknown'
            # Append Orbit to the launch_dict
            launch_dict['Orbit'].append(orbit)

            # Extract and process Customer from the seventh cell
            # Use the text in <a> tag if present
            customer = row[6].a.string if row[6].a else 'Unknown'
            # Append Customer to the launch_dict
            launch_dict['Customer'].append(customer)

            # Extract and process Launch Outcome from the eighth cell
            # Use list(row[7].strings)[0] to get the first string if available
            launch_outcome = list(row[7].strings)[0] if row[7].strings else 'Unknown'
            # Append Launch Outcome to the launch_dict
            launch_dict['Launch outcome'].append(launch_outcome)

            # Extract and process Booster Landing from the ninth cell
            # The function landing_status() should handle this extraction
            booster_landing = landing_status(row[8])  # Process the ninth <td> element
            # Append Booster Landing to the launch_dict
            launch_dict['Booster landing'].append(booster_landing if booster_landing else 'Unknown')


In [141]:
#launch_dict

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [142]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

#Cleaning up the unwanted '\n'
#The regex=True parameter allows replace() to interpret '\n' as a regex pattern.
df.replace('\n', '', regex=True, inplace=True)
df#check

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


In [143]:
df.to_csv('spacex_web_scraped.csv', index=False)

## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
